In [53]:
rela = False
# cov_str = 'Cai2011Adaptive_Model1'
cov_str = 'Cai2011Adaptive_Model2_my'
data_path = '../data_6-10/'

In [54]:
import sys, os
sys.path.append(os.getcwd() + '\..')
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

# 数据存入DataFrame

In [55]:
id1 = pd.MultiIndex.from_product([['fro', '2'], [300], [100, 300, 500]])
id2 = pd.MultiIndex.from_product([['fro', '2'], [100], [30, 100, 200]])
id = id1.append(id2)
id = id.sort_values()
df = pd.DataFrame(index = id)

### Network Guided Banding

In [56]:
simu_str = 'lx_band'

for N, T in [(30, 100), (100, 100), (200, 100)] + [(100, 300), (300, 300), (500, 300)]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        ord = str(ord)
        
        for eta in [1, 0.8, 0.5, 0.1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            # print(file_name)
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, T, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 1] ['lx_band', 0.8] ['lx_band', 0.5] ['lx_band', 0.1]
2   100 30      3.88(1.00)       4.28(1.07)       5.14(1.19)       7.58(0.55)
        100     7.97(1.84)       8.10(1.72)       8.97(1.06)       9.73(0.21)
        200     9.99(0.55)      10.00(0.51)      10.07(0.20)      10.11(0.09)
    300 100     3.40(0.26)       3.52(0.29)       4.99(0.34)       8.62(0.53)
        300     3.65(0.16)       3.69(0.17)       5.07(0.22)       8.97(0.36)
        500     3.97(0.15)       4.04(0.14)       4.54(0.14)       8.70(0.30)
fro 100 30      7.26(0.75)       7.81(0.71)       8.88(0.61)      10.33(0.24)
        100    19.51(1.73)      19.67(1.55)      20.59(0.71)      21.14(0.18)
        200    30.37(0.49)      30.37(0.48)      30.44(0.23)      30.48(0.16)
    300 100     9.97(0.29)      10.87(0.32)      14.60(0.34)      19.54(0.51)
        300    17.05(0.32)      18.60(0.31)      24.27(0.31)      33.76(0.58)
        500    20.43(0.32)      22.08(0.32)      29.92(0.41)      43.17(0.73)

### Network Guided Thresholding

In [57]:
simu_str = 'LSRthreshold'
num_cv = 50

for N, T in [(30, 100), (100, 100), (200, 100)] + [(100, 300), (300, 300), (500, 300)]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        ord = str(ord)
        
        for l in np.linspace(0, 1, 6):
            for prob in [0.5, 0.8, 1]:
                for qrob in [0, 0.1, 0.2]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, l, prob, qrob, 'brute', num_cv]
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        print('File not found.')
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    if not rela:
                        data *= normS # absolute error
                    df[col][ord, T, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 1] ['lx_band', 0.8] ['lx_band', 0.5] ['lx_band', 0.1]  \
2   100 30      3.88(1.00)       4.28(1.07)       5.14(1.19)       7.58(0.55)   
        100     7.97(1.84)       8.10(1.72)       8.97(1.06)       9.73(0.21)   
        200     9.99(0.55)      10.00(0.51)      10.07(0.20)      10.11(0.09)   
    300 100     3.40(0.26)       3.52(0.29)       4.99(0.34)       8.62(0.53)   
        300     3.65(0.16)       3.69(0.17)       5.07(0.22)       8.97(0.36)   
        500     3.97(0.15)       4.04(0.14)       4.54(0.14)       8.70(0.30)   
fro 100 30      7.26(0.75)       7.81(0.71)       8.88(0.61)      10.33(0.24)   
        100    19.51(1.73)      19.67(1.55)      20.59(0.71)      21.14(0.18)   
        200    30.37(0.49)      30.37(0.48)      30.44(0.23)      30.48(0.16)   
    300 100     9.97(0.29)      10.87(0.32)      14.60(0.34)      19.54(0.51)   
        300    17.05(0.32)      18.60(0.31)      24.27(0.31)      33.76(0.58)   
        500    20.43(0.32)      22.08(0.32)      29.92(0.41)      43.17(0.73)   

            ['LSRthreshold', 0.0, 0.5, 0, 'brute', 50]  \
2   100 30                                  4.00(0.67)   
        100                                 8.03(1.16)   
        200                                11.22(2.30)   
    300 100                                 4.88(0.39)   
        300                                 9.16(0.42)   
        500                                11.61(0.30)   
fro 100 30                                  8.46(0.66)   
        100                                26.41(2.65)   
        200                                47.11(7.44)   
    300 100                                17.95(0.51)   
        300                                50.27(0.76)   
        500                                76.97(0.89)   

            ['LSRthreshold', 0.0, 0.5, 0.1, 'brute', 50]  \
2   100 30                                    4.15(0.67)   
        100                                   8.70(1.15)   
        200                                  12.63(2.15)   
    300 100                                   5.14(0.40)   
        300                                   9.90(0.43)   
        500                                  12.74(0.30)   
fro 100 30                                    8.92(0.62)   
        100                                  29.16(2.19)   
        200                                  54.72(5.71)   
    300 100                                  19.07(0.50)   
        300                                  54.98(0.70)   
        500                                  85.68(0.81)   

            ['LSRthreshold', 0.0, 0.5, 0.2, 'brute', 50]  \
2   100 30                                    4.30(0.69)   
        100                                   9.34(1.18)   
        200                                  14.04(2.03)   
    300 100                                   5.39(0.41)   
        300                                  10.63(0.43)   
        500                                  13.83(0.31)   
fro 100 30                                    9.35(0.61)   
        100                                  31.63(1.95)   
        200                                  61.38(4.66)   
    300 100                                  20.11(0.48)   
        300                                  59.29(0.64)   
        500                                  93.48(0.74)   

            ['LSRthreshold', 0.0, 0.8, 0, 'brute', 50]  \
2   100 30                                  3.74(0.73)   
        100                                 6.93(1.51)   
        200                                 9.74(2.24)   
    300 100                                 4.56(0.40)   
        300                                 8.37(0.42)   
        500                                10.52(0.31)   
fro 100 30                                  7.67(0.87)   
        100                                22.38(3.78)   
        200                                40.30(7.70)   
    300 100                                16.47(0.56)   
   

### 其它方法

In [58]:
simu_str = '' # TBD

for N, T in [(30, 100), (100, 100), (200, 100)] + [(100, 300), (300, 300), (500, 300)]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        ord = str(ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']: 
            # data_path = data_path2 if 'Threshold' in simu_str else data_path1
            # print(data_path, simu_str)
 
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, T, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 1] ['lx_band', 0.8] ['lx_band', 0.5] ['lx_band', 0.1]  \
2   100 30      3.88(1.00)       4.28(1.07)       5.14(1.19)       7.58(0.55)   
        100     7.97(1.84)       8.10(1.72)       8.97(1.06)       9.73(0.21)   
        200     9.99(0.55)      10.00(0.51)      10.07(0.20)      10.11(0.09)   
    300 100     3.40(0.26)       3.52(0.29)       4.99(0.34)       8.62(0.53)   
        300     3.65(0.16)       3.69(0.17)       5.07(0.22)       8.97(0.36)   
        500     3.97(0.15)       4.04(0.14)       4.54(0.14)       8.70(0.30)   
fro 100 30      7.26(0.75)       7.81(0.71)       8.88(0.61)      10.33(0.24)   
        100    19.51(1.73)      19.67(1.55)      20.59(0.71)      21.14(0.18)   
        200    30.37(0.49)      30.37(0.48)      30.44(0.23)      30.48(0.16)   
    300 100     9.97(0.29)      10.87(0.32)      14.60(0.34)      19.54(0.51)   
        300    17.05(0.32)      18.60(0.31)      24.27(0.31)      33.76(0.58)   
        500    20.43(0.32)      22.08(0.32)      29.92(0.41)      43.17(0.73)   

            ['LSRthreshold', 0.0, 0.5, 0, 'brute', 50]  \
2   100 30                                  4.00(0.67)   
        100                                 8.03(1.16)   
        200                                11.22(2.30)   
    300 100                                 4.88(0.39)   
        300                                 9.16(0.42)   
        500                                11.61(0.30)   
fro 100 30                                  8.46(0.66)   
        100                                26.41(2.65)   
        200                                47.11(7.44)   
    300 100                                17.95(0.51)   
        300                                50.27(0.76)   
        500                                76.97(0.89)   

            ['LSRthreshold', 0.0, 0.5, 0.1, 'brute', 50]  \
2   100 30                                    4.15(0.67)   
        100                                   8.70(1.15)   
        200                                  12.63(2.15)   
    300 100                                   5.14(0.40)   
        300                                   9.90(0.43)   
        500                                  12.74(0.30)   
fro 100 30                                    8.92(0.62)   
        100                                  29.16(2.19)   
        200                                  54.72(5.71)   
    300 100                                  19.07(0.50)   
        300                                  54.98(0.70)   
        500                                  85.68(0.81)   

            ['LSRthreshold', 0.0, 0.5, 0.2, 'brute', 50]  \
2   100 30                                    4.30(0.69)   
        100                                   9.34(1.18)   
        200                                  14.04(2.03)   
    300 100                                   5.39(0.41)   
        300                                  10.63(0.43)   
        500                                  13.83(0.31)   
fro 100 30                                    9.35(0.61)   
        100                                  31.63(1.95)   
        200                                  61.38(4.66)   
    300 100                                  20.11(0.48)   
        300                                  59.29(0.64)   
        500                                  93.48(0.74)   

            ['LSRthreshold', 0.0, 0.8, 0, 'brute', 50]  \
2   100 30                                  3.74(0.73)   
        100                                 6.93(1.51)   
        200                                 9.74(2.24)   
    300 100                                 4.56(0.40)   
        300                                 8.37(0.42)   
        500                                10.52(0.31)   
fro 100 30                                  7.67(0.87)   
        100                                22.38(3.78)   
        200                                40.30(7.70)   
    300 100                                16.47(0.56)   
   

### 修改表头

In [59]:
np.allclose(1, 1.05, atol=0.1)

True

In [60]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Network Guided Banding ($\eta={eta}$)'
    elif 'LSRthreshold' in col or 'thG' in col:
        formal_name = {'LSRthreshold': 'Network Guided Thresholding', 'thG': 'Threshold then Augment'}[col[0]]
        l, p, q = col[1:-2]

        l = 0.6 if np.allclose(l, 0.6, atol = 0.01) else l # np.linspace 带来的精度误差

        new_col = fr'{formal_name} ($(l, p, q)={(l, p, q)}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols
df.head(1)

,,,Network Guided Banding ($\eta=1$),Network Guided Banding ($\eta=0.8$),Network Guided Banding ($\eta=0.5$),Network Guided Banding ($\eta=0.1$),"Network Guided Thresholding ($(l, p, q)=(0.0, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.0, 1, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.2, 1, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.4, 1, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.6, 1, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(0.8, 1, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.5, 0)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.5, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.5, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.8, 0)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.8, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 0.8, 0.2)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 1, 0)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 1, 0.1)$)","Network Guided Thresholding ($(l, p, q)=(1.0, 1, 0.2)$)",Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink
2,100,30,3.88(1.00),4.28(1.07),5.14(1.19),7.58(0.55),4.00(0.67),4.15(0.67),4.30(0.69),3.74(0.73),3.91(0.73),4.09(0.74),3.14(0.76),3.36(0.74),3.53(0.73),4.00(0.67),4.15(0.67),4.30(0.69),3.74(0.73),3.91(0.73),4.09(0.74),3.14(0.76),3.36(0.74),3.53(0.73),4.00(0.67),4.15(0.67),4.30(0.69),3.74(0.73),3.91(0.73),4.09(0.74),3.14(0.76),3.36(0.74),3.53(0.73),4.00(0.67),4.15(0.67),4.30(0.69),3.74(0.73),3.91(0.73),4.09(0.74),3.14(0.76),3.36(0.74),3.53(0.73),4.00(0.67),4.15(0.67),4.30(0.69),3.74(0.73),3.91(0.73),4.09(0.74),3.14(0.76),3.36(0.74),3.53(0.73),4.27(0.64),4.35(0.65),4.44(0.66),4.27(0.64),4.35(0.65),4.44(0.66),4.27(

In [61]:
id1 = pd.MultiIndex.from_product([['Frobenius Norm', 'Matrix-2 Norm'], [300], [100, 300, 500]])
id2 = pd.MultiIndex.from_product([['Frobenius Norm', 'Matrix-2 Norm'], [100], [30, 100, 200]])
new_id = id1.append(id2)
new_id = new_id.sort_values()
df.index = new_id
df1 = df.reset_index()
df1.rename(mapper = {'level_0': 'Norm', 'level_1': '$N$'}, axis = 1, inplace = True)
df1.set_index(keys = ['Norm', '$N$'], inplace = True)
df = df1

In [62]:
df

level_2 Network Guided Banding ($\eta=1$)  \
Norm           $N$                                              
Frobenius Norm 100       30                        3.88(1.00)   
               100      100                        7.97(1.84)   
               100      200                        9.99(0.55)   
               300      100                        3.40(0.26)   
               300      300                        3.65(0.16)   
               300      500                        3.97(0.15)   
Matrix-2 Norm  100       30                        7.26(0.75)   
               100      100                       19.51(1.73)   
               100      200                       30.37(0.49)   
               300      100                        9.97(0.29)   
               300      300                       17.05(0.32)   
               300      500                       20.43(0.32)   

                   Network Guided Banding ($\eta=0.8$)  \
Norm           $N$                                       
Frobenius Norm 100                          4.28(1.07)   
               100                          8.10(1.72)   
               100                         10.00(0.51)   
               300                          3.52(0.29)   
               300                          3.69(0.17)   
               300                          4.04(0.14)   
Matrix-2 Norm  100                          7.81(0.71)   
               100                         19.67(1.55)   
               100                         30.37(0.48)   
               300                         10.87(0.32)   
               300                         18.60(0.31)   
               300                         22.08(0.32)   

                   Network Guided Banding ($\eta=0.5$)  \
Norm           $N$                                       
Frobenius Norm 100                          5.14(1.19)   
               100                          8.97(1.06)   
               100                         10.07(0.20)   
               300                          4.99(0.34)   
               300                          5.07(0.22)   
               300                          4.54(0.14)   
Matrix-2 Norm  100                          8.88(0.61)   
               100                         20.59(0.71)   
               100                         30.44(0.23)   
               300                         14.60(0.34)   
               300                         24.27(0.31)   
               300                         29.92(0.41)   

                   Network Guided Banding ($\eta=0.1$)  \
Norm           $N$                                       
Frobenius Norm 100                          7.58(0.55)   
               100                          9.73(0.21)   
               100                         10.11(0.09)   
               300                          8.62(0.53)   
               300                          8.97(0.36)   
               300                          8.70(0.30)   
Matrix-2 Norm  100                         10.33(0.24)   
               100                         21.14(0.18)   
               100                         30.48(0.16)   
               300                         19.54(0.51)   
               300                         33.76(0.58)   
               300                         43.17(0.73)   

                   Network Guided Thresholding ($(l, p, q)=(0.0, 0.5, 0)$)  \
Norm           $N$                                                           
Frobenius Norm 100                                         4.00(0.67)        
               100                                         8.03(1.16)        
               100                                        11.22(2.30)        
               300                                         4.88(0.39)        
               300                                         9.16(0.42)        
               300                                        11.61(0.30)        
Matrix-2 Norm  100                            

# 存为csv格式

In [63]:
df.to_csv(path_or_buf = cov_str + '.csv')

# 存为latex格式

In [64]:
buf = '../latex/' + cov_str + '.tex'
model_label = 1 if cov_str == 'Cai2011Adaptive_Model1' else 2
caption = f'Simulation Results of Model {model_label}'

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'l' * len(new_id[0]) + '|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = f'res-model{model_label}', 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_10040\1733114020.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [65]:
pd.get_option('display.width')

80

In [66]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

# 横排表
lines[0] = r'\begin{sidewaystable}[htbp]' + '\n'
lines[-1] = r'\end{sidewaystable}' + '\n'

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
